In [1]:
from submodules import Objects, Robots, Tasks, Users
from submodules.Scenes import Scene
from submodules.Actions import Actions
from submodules.Gestures import Gestures
from submodules.Users import Users
from model_noK import *
import numpy as np
from numpy import array as a

Firstly, let's go through iteration cycle
## Create scene

In [2]:
s = Scene(init='drawer_and_cups')
s.cup1.position = s.r.eef_position

Configuration of gestures and actions

In [3]:
A = Actions.A
G = Gestures.G
U = Users.U
UID = Users.selected_id
print(A)
print(G)
print(U)
print(UID)

['move_up', 'open', 'put', 'pour', 'close', 'pick_up']
['swipe_up', 'open', 'swipe_back', 'hold', 'rotate', 'swipe_forward']
['Jan', 'Mara']
0


## Generate policy

In [4]:
# Static parmeters
TaTo = ('open', 'drawer')
scene_state = s.generate_scene_state(A, G, U, UID, TaTo)

bn = MappingBayesNet(scene_state)

print(bn.create_observation())
bn.init_policy_simple()

{'focus_point': array([[2.15170773, 0.01087358, 0.01004739]]), 'gesture_vec': array([0.07318218, 0.64108303, 0.07559044, 0.05414416, 0.1291822 ,
       0.02681798])}


{'CM_est': array([[1, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0],
        [0, 0, 1, 0, 0, 0],
        [0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 0, 1]]),
 'i': 'init'}

## Create target state

In [5]:
s2 = Scene(init='from_dict', import_data=scene_state)
s2.drawer.open()
s2.drawer.put_in(s2.cup1)
s2.cup1.gripper_move(s2.drawer.position)
s2.object_positions

[array([2, 0, 0]), array([1, 3, 3]), array([2, 2, 0])]

## Select action

In [6]:
action = bn.select_action()
action

{'target_object': 'drawer', 'target_action': 'open'}

## New observation / new world state
- Updates the scene


In [7]:
# temporary place
Actions.do(s, action, ignore_location=True)
s.drawer.info

drawer, drawer, [2 0 0], opened, []


# Reward computation

In [8]:
s == s2

7.0

# Easy example: Open drawer, put there a cup

In [9]:
action_sequence = Tasks.CupToDrawer.action_sequence
for action_input in action_sequence:
    scene_state = s.generate_scene_state(A, G, U, UID, TaTo)
    scene_state['TA'] = action_input[0]
    scene_state['TO'] = action_input[1]
    
    bn = MappingBayesNet(scene_state)
    bn.create_observation()
    bn.init_policy_simple()
    action = bn.select_action()
    print(action)
    
    Actions.do(s, action)
    
    reward = s == s2
    print(f"Reward: {reward}")


{'target_object': 'drawer', 'target_action': 'open'}
Reward: 7.0
{'target_object': 'cup1', 'target_action': 'pick_up'}
Reward: 7.0
{'target_object': 'drawer', 'target_action': 'put'}
Reward: 7.0


# Generating observations

In [10]:
s = Scene(init='drawer_and_cups', random=True)
print(s)

Scene info. shape: [4, 4, 4]
drawer, drawer, [2 0 0], closed, []
cup1, cup, [2 1 0], full
cup2, cup, [2 2 0], empty
Robot: [1 2 3], opened, rotation: 0, attached: -


In [11]:
s = Scene(init='drawer_and_cups', random=False)
print(s)

Scene info. shape: [4, 4, 4]
drawer, drawer, [2 0 0], closed, []
cup1, cup, [2 1 0], empty
cup2, cup, [2 2 0], empty
Robot: [0 0 3], opened, rotation: 0, attached: -


# Apply actions on scene

In [12]:
Actions.do(s, ('open', 'drawer'))  
print(s)

Scene info. shape: [4, 4, 4]
drawer, drawer, [2 0 0], closed, []
cup1, cup, [2 1 0], empty
cup2, cup, [2 2 0], empty
Robot: [0 0 3], opened, rotation: 0, attached: -


In [13]:
Actions.do(s, ('close', 'drawer'))
print(s)

Scene info. shape: [4, 4, 4]
drawer, drawer, [2 0 0], closed, []
cup1, cup, [2 1 0], empty
cup2, cup, [2 2 0], empty
Robot: [0 0 3], opened, rotation: 0, attached: -


In [14]:
Actions.do(s, ('pick_up', 'cup1'))
print(s)

Scene info. shape: [4, 4, 4]
drawer, drawer, [2 0 0], closed, []
cup1, cup, [2 1 0], empty
cup2, cup, [2 2 0], empty
Robot: [0 0 3], opened, rotation: 0, attached: -


In [15]:
A

['move_up', 'open', 'put', 'pour', 'close', 'pick_up']

In [16]:
G

['swipe_up', 'open', 'swipe_back', 'hold', 'rotate', 'swipe_forward']

In [17]:
s.O

['drawer', 'cup1', 'cup2']

In [18]:
Actions.do(s, ('put', 'drawer'))
print(s)

Scene info. shape: [4, 4, 4]
drawer, drawer, [2 0 0], closed, []
cup1, cup, [2 1 0], empty
cup2, cup, [2 2 0], empty
Robot: [0 0 3], opened, rotation: 0, attached: -


In [19]:
Actions.do(s, ('open', 'drawer'))
print(s)

Scene info. shape: [4, 4, 4]
drawer, drawer, [2 0 0], closed, []
cup1, cup, [2 1 0], empty
cup2, cup, [2 2 0], empty
Robot: [0 0 3], opened, rotation: 0, attached: -


In [20]:
Actions.do(s, ('put', 'drawer'))
print(s)

Scene info. shape: [4, 4, 4]
drawer, drawer, [2 0 0], closed, []
cup1, cup, [2 1 0], empty
cup2, cup, [2 2 0], empty
Robot: [0 0 3], opened, rotation: 0, attached: -


In [21]:
Actions.do(s, ('pick_up', 'cup2'))
print(s)

Scene info. shape: [4, 4, 4]
drawer, drawer, [2 0 0], closed, []
cup1, cup, [2 1 0], empty
cup2, cup, [2 2 0], empty
Robot: [0 0 3], opened, rotation: 0, attached: -


In [22]:
Actions.do(s, ('put', 'drawer'))
print(s)

Scene info. shape: [4, 4, 4]
drawer, drawer, [2 0 0], closed, []
cup1, cup, [2 1 0], empty
cup2, cup, [2 2 0], empty
Robot: [0 0 3], opened, rotation: 0, attached: -


In [23]:
s = Scene(init='drawer,cup,cup')
print(s)

Scene info. shape: [4, 4, 4]
drawer, drawer, [1 2 0], opened, []
cup, cup, [3 1 0], full
cup1, cup, [2 3 0], empty
Robot: [1 0 3], closed, rotation: 1, attached: -


In [24]:
Actions.do(s, ('pick_up', 'cup'))

False

In [25]:
Actions.do(s, ('open', 'drawer'))

False

In [26]:
Actions.do(s, ('put', 'drawer'))

False

In [27]:
Actions.do(s, ('close', 'drawer'))

False

In [28]:
print(s)

Scene info. shape: [4, 4, 4]
drawer, drawer, [1 2 0], opened, []
cup, cup, [3 1 0], full
cup1, cup, [2 3 0], empty
Robot: [1 0 3], closed, rotation: 1, attached: -


# Move tests

In [29]:
s = Scene(init='cup')
print(s)

Scene info. shape: [4, 4, 4]
cup, cup, [2 2 0], full
Robot: [3 3 3], closed, rotation: 0, attached: -


In [30]:
s.r.eef_position = s.cup.position

In [31]:
Actions.do(s, ('pick_up','cup'))
s.info

Scene info. shape: [4, 4, 4]
cup, cup, [2 2 0], full
Robot: [2 2 0], closed, rotation: 0, attached: cup


In [32]:
s = Scene(init='cup,cup')
s.r.eef_position = s.cup.position
Actions.do(s, ('pick_up', 'cup'))

True

In [33]:
s.info

Scene info. shape: [4, 4, 4]
cup, cup, [3 0 0], full
cup1, cup, [3 1 0], full
Robot: [3 0 0], closed, rotation: 0, attached: cup


In [34]:
Actions.do(s, ('move_up',None))

True

In [35]:
s.info

Scene info. shape: [4, 4, 4]
cup, cup, [3 0 1], full
cup1, cup, [3 1 0], full
Robot: [3 0 1], closed, rotation: 0, attached: cup


# Generating data

In [36]:
# Generate random scene
scenes = []
scenes.append(Scene(init='drawer,cup,cup'))
s = scenes[0]
scenes[0].info
s.O

Scene info. shape: [4, 4, 4]
drawer, drawer, [1 1 0], closed, []
cup, cup, [2 0 0], empty
cup1, cup, [1 2 0], full
Robot: [1 3 3], closed, rotation: 0, attached: -


['drawer', 'cup', 'cup1']

In [37]:
(Actions.get_random_action(), s.get_random_object())

('move_up', 'cup1')

In [38]:
TaTos = []
# Generate random action
for i in range(10):
    while True:
        TaTo = (Actions.get_random_action(), s.get_random_object())
        if Actions.do(s, TaTo, ignore_location=True):
            TaTos.append(TaTo)
            break

In [39]:
TaTos

[('pick_up', 'cup'),
 ('close', 'drawer'),
 ('open', 'drawer'),
 ('close', 'drawer'),
 ('close', 'drawer'),
 ('close', 'drawer'),
 ('open', 'drawer'),
 ('put', 'drawer'),
 ('open', 'drawer'),
 ('pick_up', 'cup1')]

In [40]:
s.info

Scene info. shape: [4, 4, 4]
drawer, drawer, [1 1 0], opened, ['cup']
cup, cup, [2 0 0], empty
cup1, cup, [1 2 0], full
Robot: [1 3 3], closed, rotation: 0, attached: cup1


In [41]:
s.cup.print_structure()

Structure:
[cup]


In [42]:
s.cup1.print_structure()

Structure:
[cup1]


In [43]:
TaTo = (Actions.get_random_action(), s.get_random_object())
Actions.do(s, TaTo, ignore_location=True)

False

In [44]:
TaTo = ('pick_up', 'cup')
Actions.do(s, TaTo, ignore_location=True)

False

In [45]:
s.info

Scene info. shape: [4, 4, 4]
drawer, drawer, [1 1 0], opened, ['cup']
cup, cup, [2 0 0], empty
cup1, cup, [1 2 0], full
Robot: [1 3 3], closed, rotation: 0, attached: cup1


In [46]:
TaTo = ('put_on_target', 'cup1')
Actions.do(s, TaTo, ignore_location=True)

True

In [47]:
TaTos = []
# Generate random action
for i in range(10):
    j = 0
    while True:
        TaTo = (Actions.get_action(j//len(A)), s.get_object(j%len(s.O)))
        if Actions.do(s, TaTo, ignore_location=True):
            TaTos.append(TaTo)
            break
        j += 1
        if j == 64: raise Exception("out of options")
            

In [48]:
s.cup1.fill()
s1 = s.copy()

In [49]:
s.info

Scene info. shape: [4, 4, 4]
drawer, drawer, [1 1 0], opened, ['cup']
cup, cup, [2 0 0], empty
cup1, cup, [2 3 0], full
Robot: [1 3 3], closed, rotation: 0, attached: -


In [50]:
s1.info

Scene info. shape: [4, 4, 4]
drawer, drawer, [1 1 0], opened, ['cup']
cup, cup, [2 0 0], empty
cup1, cup, [2 3 0], full
Robot: [1 3 3], closed, rotation: 0, attached: -


# Attempt 1: Loop with reward

- prototype
- written notes and suggestions

In [51]:
# Static params
A = Actions.A
G = Gestures.G
U = Users.U
UID = Users.selected_id

# hyperparameters
alpha = 0.1
gamma = 0.6
epsilon = 0.1

In [52]:
# Task
t = Tasks.CupToDrawer()
# train loops 
n = 2
for i in range(1,n+1):
    epochs, penalties, reward, = 0, 0, 0
    done = False
    
    # Init. task start scene 's', target scene 's2'
    s, s2 = t.get_start_target_scenes()
    
    # Generate first scene state
    TaTo_User = t.decide(s)
    state = s.generate_scene_state(A, G, U, UID, TaTo_User)
    bn = MappingBayesNet(state)

    while not done:
        # Exploring actions space x Using learned values
        if np.random.uniform(0, 1) < epsilon:
            # There should be: Sampling from env/scene (p=10%)
            '''
            action = scene.sample()
            '''
            TaTo_action = t.decide(s)
        else:
            # There should be: Pick action using learned values (p=90%)
            '''
            action = np.argmax(q_table[state])
            '''
            TaTo_action = t.decide(s)
        
        
        print(f"Action: {TaTo_action}")
        # Apply action, compute reward, check if done
        s, reward, done = Actions.step(s, s2, TaTo_action, ignore_location=True)
        # Update policy based on reward
        bn.policy_update(reward)

Action: ('open', 'drawer')
Action: ('pick_up', 'cup')
Action: ('put', 'drawer')
Action: ('open', 'drawer')
Action: ('pick_up', 'cup')
Action: ('put', 'drawer')


In [53]:
s == s2

True

In [54]:
s.info

Scene info. shape: [4, 4, 4]
cup, cup, [3 0 0], empty
drawer, drawer, [1 1 0], opened, ['cup']
Robot: [0 0 3], opened, rotation: 0, attached: -


In [55]:
s2.info

Scene info. shape: [4, 4, 4]
cup, cup, [3 0 0], empty
drawer, drawer, [1 1 0], opened, ['cup']
Robot: [0 0 3], opened, rotation: 0, attached: -


In [56]:
s.drawer.contains_list

['cup']

In [57]:
bn.policy['CM_est']

array([[0, 1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0],
       [0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 1]])

## Train the policy

In [96]:
#A = Actions.A = ['move_up', 'move_down', 'move_left', 'move_right', 'open', 'close']
policy = None
t = Tasks.CupToDrawer()

for i in range(1,2):
    epochs, penalties, reward, = 0, 0, 0
    done = False

    s, s2 = t.get_start_target_scenes()
    
    TaTo_User = t.decide(s)
    state = s.generate_scene_state(A, G, U, UID, TaTo_User)
    bn = MappingBayesNet(state, policy=policy)
    
    while not done:
        TaTo_User = t.decide(s)
        bn.create_observation(TaTo_User)
        action = bn.select_action()
        s, reward, done = Actions.step(s, s2, action, ignore_location=True, out=False)
        bn.policy_update(reward)
    print(list(zip(bn.reward_history, bn.action_history)))
    policy = bn.policy

[(5.0, {'target_object': 'drawer', 'target_action': 'open'}), (5.0, {'target_object': 'cup', 'target_action': 'move_up'}), (5.0, {'target_object': 'cup', 'target_action': 'move_up'}), (5.0, {'target_object': 'cup', 'target_action': 'pick_up'}), (4.0, {'target_object': 'drawer', 'target_action': 'close'}), (4.0, {'target_object': 'drawer', 'target_action': 'put'}), (4.0, {'target_object': 'drawer', 'target_action': 'put'}), (4.0, {'target_object': 'drawer', 'target_action': 'put'}), (4.0, {'target_object': 'drawer', 'target_action': 'put'}), (4.0, {'target_object': 'drawer', 'target_action': 'pour'}), (4.0, {'target_object': 'drawer', 'target_action': 'close'}), (4.0, {'target_object': 'drawer', 'target_action': 'close'}), (4.0, {'target_object': 'drawer', 'target_action': 'close'}), (4.0, {'target_object': 'drawer', 'target_action': 'close'}), (4.0, {'target_object': 'drawer', 'target_action': 'close'}), (4.0, {'target_object': 'drawer', 'target_action': 'close'}), (4.0, {'target_objec

In [97]:
bn.policy['CM_est']

array([[1, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0],
       [0, 1, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1, 0]])

In [98]:
bn.reward_history

[5.0,
 5.0,
 5.0,
 5.0,
 4.0,
 4.0,
 4.0,
 4.0,
 4.0,
 4.0,
 4.0,
 4.0,
 4.0,
 4.0,
 4.0,
 4.0,
 4.0,
 4.0,
 4.0,
 4.0,
 4.0,
 4.0,
 4.0,
 4.0,
 4.0,
 4.0,
 4.0,
 4.0,
 4.0,
 4.0,
 4.0,
 4.0,
 4.0,
 4.0,
 4.0,
 4.0,
 4.0,
 4.0,
 4.0,
 4.0,
 4.0,
 4.0,
 4.0,
 4.0,
 4.0,
 4.0,
 4.0,
 4.0,
 4.0,
 4.0,
 4.0,
 4.0,
 4.0,
 4.0,
 4.0,
 4.0,
 4.0,
 4.0,
 4.0,
 4.0,
 4.0,
 4.0,
 4.0,
 5.0,
 4.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 5.0,
 True]

## Test the policy
- It trained a false policy

In [95]:
epochs, penalties, reward, = 0, 0, 0
done = False

s, s2 = t.get_start_target_scenes()
    
TaTo_User = t.decide(s)
state = s.generate_scene_state(A, G, U, UID, TaTo_User)
bn = MappingBayesNet(state, policy=policy)

steps = 0
while not done:
    TaTo_action = t.decide(s)
    bn.create_observation(TaTo_action)
    action = bn.select_action()
    s, reward, done = Actions.step(s, s2, action, ignore_location=True, out=False)
    print(action, reward)
    steps += 1

{'target_object': 'drawer', 'target_action': 'open'} 5.0
{'target_object': 'cup', 'target_action': 'pour'} 5.0
{'target_object': 'cup', 'target_action': 'pour'} 5.0
{'target_object': 'cup', 'target_action': 'pour'} 5.0
{'target_object': 'cup', 'target_action': 'pour'} 5.0
{'target_object': 'cup', 'target_action': 'pour'} 5.0
{'target_object': 'cup', 'target_action': 'pour'} 5.0
{'target_object': 'cup', 'target_action': 'pour'} 5.0
{'target_object': 'cup', 'target_action': 'pour'} 5.0
{'target_object': 'cup', 'target_action': 'pour'} 5.0
{'target_object': 'cup', 'target_action': 'pour'} 5.0
{'target_object': 'cup', 'target_action': 'pour'} 5.0
{'target_object': 'cup', 'target_action': 'pour'} 5.0
{'target_object': 'cup', 'target_action': 'pour'} 5.0
{'target_object': 'cup', 'target_action': 'pour'} 5.0
{'target_object': 'cup', 'target_action': 'pour'} 5.0
{'target_object': 'cup', 'target_action': 'pour'} 5.0
{'target_object': 'cup', 'target_action': 'pour'} 5.0
{'target_object': 'cup', 

{'target_object': 'cup', 'target_action': 'pour'} 5.0
{'target_object': 'cup', 'target_action': 'pour'} 5.0
{'target_object': 'cup', 'target_action': 'pour'} 5.0
{'target_object': 'cup', 'target_action': 'pour'} 5.0
{'target_object': 'cup', 'target_action': 'pour'} 5.0
{'target_object': 'cup', 'target_action': 'pour'} 5.0
{'target_object': 'cup', 'target_action': 'pour'} 5.0
{'target_object': 'cup', 'target_action': 'pour'} 5.0
{'target_object': 'cup', 'target_action': 'pour'} 5.0
{'target_object': 'cup', 'target_action': 'pour'} 5.0
{'target_object': 'cup', 'target_action': 'pour'} 5.0
{'target_object': 'cup', 'target_action': 'pour'} 5.0
{'target_object': 'cup', 'target_action': 'pour'} 5.0
{'target_object': 'cup', 'target_action': 'pour'} 5.0
{'target_object': 'cup', 'target_action': 'pour'} 5.0
{'target_object': 'cup', 'target_action': 'pour'} 5.0
{'target_object': 'cup', 'target_action': 'pour'} 5.0
{'target_object': 'cup', 'target_action': 'pour'} 5.0
{'target_object': 'cup', 'ta

{'target_object': 'cup', 'target_action': 'pour'} 5.0
{'target_object': 'cup', 'target_action': 'pour'} 5.0
{'target_object': 'cup', 'target_action': 'pour'} 5.0
{'target_object': 'cup', 'target_action': 'pour'} 5.0
{'target_object': 'cup', 'target_action': 'pour'} 5.0
{'target_object': 'cup', 'target_action': 'pour'} 5.0
{'target_object': 'cup', 'target_action': 'pour'} 5.0
{'target_object': 'cup', 'target_action': 'pour'} 5.0
{'target_object': 'cup', 'target_action': 'pour'} 5.0
{'target_object': 'cup', 'target_action': 'pour'} 5.0
{'target_object': 'cup', 'target_action': 'pour'} 5.0
{'target_object': 'cup', 'target_action': 'pour'} 5.0
{'target_object': 'cup', 'target_action': 'pour'} 5.0
{'target_object': 'cup', 'target_action': 'pour'} 5.0
{'target_object': 'cup', 'target_action': 'pour'} 5.0
{'target_object': 'cup', 'target_action': 'pour'} 5.0
{'target_object': 'cup', 'target_action': 'pour'} 5.0
{'target_object': 'cup', 'target_action': 'pour'} 5.0
{'target_object': 'cup', 'ta

{'target_object': 'cup', 'target_action': 'pour'} 5.0
{'target_object': 'cup', 'target_action': 'pour'} 5.0
{'target_object': 'cup', 'target_action': 'pour'} 5.0
{'target_object': 'cup', 'target_action': 'pour'} 5.0
{'target_object': 'cup', 'target_action': 'pour'} 5.0
{'target_object': 'cup', 'target_action': 'pour'} 5.0
{'target_object': 'cup', 'target_action': 'pour'} 5.0
{'target_object': 'cup', 'target_action': 'pour'} 5.0
{'target_object': 'cup', 'target_action': 'pour'} 5.0
{'target_object': 'cup', 'target_action': 'pour'} 5.0
{'target_object': 'cup', 'target_action': 'pour'} 5.0
{'target_object': 'cup', 'target_action': 'pour'} 5.0
{'target_object': 'cup', 'target_action': 'pour'} 5.0
{'target_object': 'cup', 'target_action': 'pour'} 5.0
{'target_object': 'cup', 'target_action': 'pour'} 5.0
{'target_object': 'cup', 'target_action': 'pour'} 5.0
{'target_object': 'cup', 'target_action': 'pour'} 5.0
{'target_object': 'cup', 'target_action': 'pour'} 5.0
{'target_object': 'cup', 'ta

{'target_object': 'cup', 'target_action': 'pour'} 5.0
{'target_object': 'cup', 'target_action': 'pour'} 5.0
{'target_object': 'cup', 'target_action': 'pour'} 5.0
{'target_object': 'cup', 'target_action': 'pour'} 5.0
{'target_object': 'cup', 'target_action': 'pour'} 5.0
{'target_object': 'cup', 'target_action': 'pour'} 5.0
{'target_object': 'cup', 'target_action': 'pour'} 5.0
{'target_object': 'cup', 'target_action': 'pour'} 5.0
{'target_object': 'cup', 'target_action': 'pour'} 5.0
{'target_object': 'cup', 'target_action': 'pour'} 5.0
{'target_object': 'cup', 'target_action': 'pour'} 5.0
{'target_object': 'cup', 'target_action': 'pour'} 5.0
{'target_object': 'cup', 'target_action': 'pour'} 5.0
{'target_object': 'cup', 'target_action': 'pour'} 5.0
{'target_object': 'cup', 'target_action': 'pour'} 5.0
{'target_object': 'cup', 'target_action': 'pour'} 5.0
{'target_object': 'cup', 'target_action': 'pour'} 5.0
{'target_object': 'cup', 'target_action': 'pour'} 5.0
{'target_object': 'cup', 'ta

...

{'target_object': 'drawer', 'target_action': 'pick_up'} 4.0
{'target_object': 'drawer', 'target_action': 'pick_up'} 4.0
{'target_object': 'drawer', 'target_action': 'pick_up'} 4.0
{'target_object': 'drawer', 'target_action': 'pick_up'} 4.0
{'target_object': 'drawer', 'target_action': 'pick_up'} 4.0
{'target_object': 'drawer', 'target_action': 'pick_up'} 4.0
{'target_object': 'drawer', 'target_action': 'pick_up'} 4.0
{'target_object': 'drawer', 'target_action': 'pick_up'} 4.0
{'target_object': 'drawer', 'target_action': 'pick_up'} 4.0
{'target_object': 'drawer', 'target_action': 'pick_up'} 4.0
{'target_object': 'drawer', 'target_action': 'pick_up'} 4.0
{'target_object': 'drawer', 'target_action': 'pick_up'} 4.0
{'target_object': 'drawer', 'target_action': 'pick_up'} 4.0
{'target_object': 'drawer', 'target_action': 'pick_up'} 4.0
{'target_object': 'drawer', 'target_action': 'pick_up'} 4.0
{'target_object': 'drawer', 'target_action': 'pick_up'} 4.0
{'target_object': 'drawer', 'target_acti

{'target_object': 'drawer', 'target_action': 'pick_up'} 4.0
{'target_object': 'drawer', 'target_action': 'pick_up'} 4.0
{'target_object': 'drawer', 'target_action': 'pick_up'} 4.0
{'target_object': 'drawer', 'target_action': 'pick_up'} 4.0
{'target_object': 'drawer', 'target_action': 'pick_up'} 4.0
{'target_object': 'drawer', 'target_action': 'pick_up'} 4.0
{'target_object': 'drawer', 'target_action': 'pick_up'} 4.0
{'target_object': 'drawer', 'target_action': 'pick_up'} 4.0
{'target_object': 'drawer', 'target_action': 'pick_up'} 4.0
{'target_object': 'drawer', 'target_action': 'pick_up'} 4.0
{'target_object': 'drawer', 'target_action': 'pick_up'} 4.0
{'target_object': 'drawer', 'target_action': 'pick_up'} 4.0
{'target_object': 'drawer', 'target_action': 'pick_up'} 4.0
{'target_object': 'drawer', 'target_action': 'pick_up'} 4.0
{'target_object': 'drawer', 'target_action': 'pick_up'} 4.0
{'target_object': 'drawer', 'target_action': 'pick_up'} 4.0
{'target_object': 'drawer', 'target_acti

{'target_object': 'drawer', 'target_action': 'pick_up'} 4.0
{'target_object': 'drawer', 'target_action': 'pick_up'} 4.0
{'target_object': 'drawer', 'target_action': 'pick_up'} 4.0
{'target_object': 'drawer', 'target_action': 'pick_up'} 4.0
{'target_object': 'drawer', 'target_action': 'pick_up'} 4.0
{'target_object': 'drawer', 'target_action': 'pick_up'} 4.0
{'target_object': 'drawer', 'target_action': 'pick_up'} 4.0
{'target_object': 'drawer', 'target_action': 'pick_up'} 4.0
{'target_object': 'drawer', 'target_action': 'pick_up'} 4.0
{'target_object': 'drawer', 'target_action': 'pick_up'} 4.0
{'target_object': 'drawer', 'target_action': 'pick_up'} 4.0
{'target_object': 'drawer', 'target_action': 'pick_up'} 4.0
{'target_object': 'drawer', 'target_action': 'pick_up'} 4.0
{'target_object': 'drawer', 'target_action': 'pick_up'} 4.0
{'target_object': 'drawer', 'target_action': 'pick_up'} 4.0
{'target_object': 'drawer', 'target_action': 'pick_up'} 4.0
{'target_object': 'drawer', 'target_acti

{'target_object': 'drawer', 'target_action': 'pick_up'} 4.0
{'target_object': 'drawer', 'target_action': 'pick_up'} 4.0
{'target_object': 'drawer', 'target_action': 'pick_up'} 4.0
{'target_object': 'drawer', 'target_action': 'pick_up'} 4.0
{'target_object': 'drawer', 'target_action': 'pick_up'} 4.0
{'target_object': 'drawer', 'target_action': 'pick_up'} 4.0
{'target_object': 'drawer', 'target_action': 'pick_up'} 4.0
{'target_object': 'drawer', 'target_action': 'pick_up'} 4.0
{'target_object': 'drawer', 'target_action': 'pick_up'} 4.0
{'target_object': 'drawer', 'target_action': 'pick_up'} 4.0
{'target_object': 'drawer', 'target_action': 'pick_up'} 4.0
{'target_object': 'drawer', 'target_action': 'pick_up'} 4.0
{'target_object': 'drawer', 'target_action': 'pick_up'} 4.0
{'target_object': 'drawer', 'target_action': 'pick_up'} 4.0
{'target_object': 'drawer', 'target_action': 'pick_up'} 4.0
{'target_object': 'drawer', 'target_action': 'pick_up'} 4.0
{'target_object': 'drawer', 'target_acti

{'target_object': 'drawer', 'target_action': 'pick_up'} 5.0
{'target_object': 'drawer', 'target_action': 'pick_up'} 5.0
{'target_object': 'drawer', 'target_action': 'pick_up'} 5.0
{'target_object': 'drawer', 'target_action': 'pick_up'} 5.0
{'target_object': 'drawer', 'target_action': 'pick_up'} 5.0
{'target_object': 'drawer', 'target_action': 'pick_up'} 5.0
{'target_object': 'drawer', 'target_action': 'pick_up'} 5.0
{'target_object': 'drawer', 'target_action': 'pick_up'} 5.0
{'target_object': 'drawer', 'target_action': 'pick_up'} 5.0
{'target_object': 'drawer', 'target_action': 'pick_up'} 5.0
{'target_object': 'drawer', 'target_action': 'pick_up'} 5.0
{'target_object': 'drawer', 'target_action': 'pick_up'} 5.0
{'target_object': 'drawer', 'target_action': 'pick_up'} 5.0
{'target_object': 'drawer', 'target_action': 'pick_up'} 5.0
{'target_object': 'drawer', 'target_action': 'pick_up'} 5.0
{'target_object': 'drawer', 'target_action': 'pick_up'} 5.0
{'target_object': 'drawer', 'target_acti

Policy change has no influence on the behavior 


In [79]:
bn.policy

{'CM_est': array([[0, 0, 0, 0, 0, 1],
        [0, 0, 0, 1, 0, 0],
        [1, 0, 0, 0, 0, 0],
        [0, 0, 1, 0, 0, 0],
        [0, 0, 0, 0, 1, 0],
        [0, 1, 0, 0, 0, 0]]),
 'i': 'revert'}

In [80]:
steps

14881

Policy not trained well, error in the process. Assigning default policy

In [81]:
epochs, penalties, reward, = 0, 0, 0
done = False

s, s2 = t.get_start_target_scenes()
    
TaTo_User = t.decide(s)
state = s.generate_scene_state(A, G, U, UID, TaTo_User)
bn = MappingBayesNet(state)

steps = 0
while not done:
    TaTo_action = t.decide(s)
    bn.create_observation(TaTo_action)
    action = bn.select_action()
    s, reward, done = Actions.step(s, s2, action, ignore_location=True, out=False)
    print(action, reward)
    steps += 1

{'target_object': 'drawer', 'target_action': 'open'} 5.0
{'target_object': 'cup', 'target_action': 'pick_up'} 5.0
{'target_object': 'drawer', 'target_action': 'put'} True


In [82]:
bn.policy

{'CM_est': array([[1, 0, 0, 0, 0, 0],
        [0, 1, 0, 0, 0, 0],
        [0, 0, 1, 0, 0, 0],
        [0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 0, 1]]),
 'i': 'init'}

In [83]:
steps

3

# Troubleshooting

In [67]:
policy = None
t = Tasks.CupToDrawer()

for i in range(1,2):
    epochs, penalties, reward, = 0, 0, 0
    done = False

    s, s2 = t.get_start_target_scenes()
    
    TaTo_User = t.decide(s)
    state = s.generate_scene_state(A, G, U, UID, TaTo_User)
    bn = MappingBayesNet(state, policy=policy)
    
    while not done:
        TaTo_User = t.decide(s)
        bn.create_observation(TaTo_User)
        action = bn.select_action()
        s, reward, done = Actions.step(s, s2, action, ignore_location=True, out=True)
        bn.policy_update(reward, out=True)
    print(list(zip(bn.reward_history, bn.action_history)))
    print(bn.policy_history)

('open', 'drawer') done!
('pick_up', 'cup') done!
('put', 'drawer') done!
[(5.0, {'target_object': 'drawer', 'target_action': 'open'}), (5.0, {'target_object': 'cup', 'target_action': 'pick_up'}), (True, {'target_object': 'drawer', 'target_action': 'put'})]
[{'CM_est': array([[1, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0],
       [0, 0, 0, 1, 0, 0],
       [0, 0, 0, 0, 1, 0],
       [0, 0, 0, 0, 0, 1]]), 'i': 'init'}, {'CM_est': array([[0, 0, 0, 0, 1, 0],
       [0, 1, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0],
       [0, 0, 0, 1, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1]]), 'i': 'forward'}, {'CM_est': array([[0, 0, 0, 0, 1, 0],
       [0, 1, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 1],
       [1, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0]]), 'i': 'forward'}, {'CM_est': array([[0, 0, 0, 0, 1, 0],
       [0, 1, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0],
       [0, 0, 0, 1, 0, 0],
       [1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0,

#### Prove that any other than diagonal array policy is not optimal (in this setup)

In [68]:
A

['move_up', 'open', 'put', 'pour', 'close', 'pick_up']